In [10]:
###### Connect to the google drive ########
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
###### CIFAR100 form results ########
import torch
import pandas as pd

''' Remember to change true_label_ID, form_order_ID, form_result_IDs to the ID of your own '''
true_label_ID = '1AemdxSabDnYd_XbVMcgDFeOqOe9ZG4nP' # ID of small_mammals.csv or trees.csv
form_order_ID = '1HnLVjizsWXJXxaDOqzwRussrpZ9qJ7lY' # ID of small_mammals_imageID.csv or trees_imageID.csv
form_result_IDs = ['1B7VkQeJZyq_2BY1xXneJQHA-QbDJ4N9ivftsa8iC8kM',
          '1FhDQHixCkU7cLboB_LSfMWPgPWkD7n1fU1luzl6OenI',
          '1gtXxdB3XnHs6AQ0zNy75ZCeK2RngwSL82ll7Rmirpp0',
          '1FMbgeVi1DzNMVEAqWjR-q4MtpY3inodk6K37IbBUvaI',
          '13NgMNcyIRKoKg_45CWj3Q1pEYZZ437F7aBuXxZCYUC4'] # ID of small_mammals form result or trees form result

# Get the ground-truth of the order of form questions
true_label = pd.read_csv(f'https://drive.usercontent.google.com/u/3/uc?id={true_label_ID}&export=download', sep='\t')
form_order = pd.read_csv(f'https://drive.usercontent.google.com/u/3/uc?id={form_order_ID}&export=download').drop(columns=['image_ID'])
truth_label_form_order = pd.merge(form_order, true_label, on='image_name')
clean_label = torch.tensor(truth_label_form_order['class_num'].values)

# Convert form results into dataframe
df_merge = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{form_result_IDs[0]}/export?format=csv')['請輸入您的姓名:']
for sheet_id in form_result_IDs:
  url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv'
  df = pd.read_csv(url)
  df = df.drop(columns=['時間戳記'])
  df_merge = pd.merge(df_merge, df, on='請輸入您的姓名:')
df_name = df_merge['請輸入您的姓名:']
df_merge = df_merge.drop(columns=['請輸入您的姓名:'])
df_merge = df_merge.replace(['倉鼠','老鼠','兔子','錢鼠','松鼠','楓樹','橡樹','棕櫚樹','松樹','柳樹'], [36, 50, 65, 74, 80, 47, 52, 56, 59, 96])
form_response = torch.tensor(df_merge.values)

# Count the number of correct answers for each person
correct_counts = (form_response == clean_label).sum(dim=1)
df_correct_each_person = pd.DataFrame({'Person': df_name.values,'Correct_Counts': correct_counts.tolist()})
print(df_correct_each_person)

# Save organized form results
df_merge_img_order = df_merge
df_merge_img_order.loc['img'] = form_order['image_name'].tolist()
df_merge_img_order = df_merge_img_order.reindex(index=['img'] + df_merge_img_order.index.drop('img').tolist())
df_merge_img_order.to_csv('/content/drive/MyDrive/create_forms/result.csv', header=False)

# Compute delta1, delta2, gamma1, gamma2
num_data = clean_label.shape[0]
num_class = 100
labelset = torch.zeros(num_data, num_class)
for i in range(form_response.shape[0]):
  labelset[torch.arange(num_data), form_response[i]] = 1
num_class = 5

## The average number of items contained in each candidate labels set
average_candidate_num = labelset.sum(1).mean()
## The probability of true label in candidate labels set
proportion_true_labels = ((torch.where(labelset[torch.arange(num_data), clean_label]==1, True, False)==True).sum())/num_data
print('Average candidate num:{0}'.format(average_candidate_num-1))
print('The proportion of true labels in candidate label sets:{0}'.format(proportion_true_labels))
print('partial_rate:{0}, noisy_rate:{1}'.format(((average_candidate_num-1)/(num_class-1)), (1-proportion_true_labels)))

HTTPError: HTTP Error 410: Gone

In [16]:
###### CIFAR-10N results ########
### Statistical results of the public data set CIFAR-10N
### CIFAR-10N: http://noisylabels.com/
import torch

# Read cifar-10N data
noise_label = torch.load('/content/drive/MyDrive/create_forms/forms/CIFAR-10_human.pt')
clean_label = torch.from_numpy(noise_label['clean_label'])
aggre_label = torch.from_numpy(noise_label['aggre_label'])
random_label1 = torch.from_numpy(noise_label['random_label1'])
random_label2 = torch.from_numpy(noise_label['random_label2'])
random_label3 = torch.from_numpy(noise_label['random_label3'])
worst_label = torch.from_numpy(noise_label['worse_label'])

# Compute delta1, delta2, gamma1, gamma2
num_data = clean_label.shape[0]
num_class = int(torch.max(clean_label) - torch.min(clean_label) + 1)
labelset = torch.zeros(num_data, num_class)
labelset[torch.arange(num_data), aggre_label] = 1
labelset[torch.arange(num_data), random_label1] = 1
labelset[torch.arange(num_data), random_label2] = 1
labelset[torch.arange(num_data), random_label3] = 1
labelset[torch.arange(num_data), worst_label] = 1

## The average number of items contained in each candidate labels set
average_candidate_num = labelset.sum(1).mean()
## The probability of true label in candidate labels set
proportion_true_labels = ((torch.where(labelset[torch.arange(num_data), clean_label]==1, True, False)==True).sum())/num_data
print('Average candidate num:{0}'.format(average_candidate_num-1))
print('The proportion of true labels in candidate label sets:{0}'.format(proportion_true_labels))
print('partial_rate:{0}, noisy_rate:{1}'.format(((average_candidate_num-1)/(num_class-1)), (1-proportion_true_labels)))

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [1., 0., 0.,  ..., 0., 0., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.]])
Average candidate num:0.45726001262664795
The proportion of true labels in candidate label sets:0.978659987449646
partial_rate:0.050806667655706406, noisy_rate:0.021340012550354004
